# Section 1 : Importing libraries

In [3]:
import mediapipe as mp
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

# Section 2 : Using MP Holistic 

In [4]:
mp_holistic = mp.solutions.holistic    
mp_drawing = mp.solutions.drawing_utils 
mp_face_mesh = mp.solutions.face_mesh 


MediaPipe Holistic is a tool that tracks the whole human body, including:

    Body Pose: Tracks 33 keypoints on the body (e.g., head, arms, legs).
    Face: Tracks 468 detailed points on the face (e.g., eyes, mouth, nose).
    Hands: Tracks 21 keypoints on each hand.

MediaPipe drawing_utils : 

    is a tool that helps you draw points and lines on images or video frames to show the landmarks (like body joints, face features, or hand points) detected by MediaPipe models.

In [5]:
# Images will be provided from opencv in the form of Frames
# Why to convert from BGR to RGB : If the conversion is skipped, MediaPipe might produce incorrect outputs, as its models and pipelines # # are trained and optimized for RGB images.

def mediapipe_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)  # color conversion from bgr to rgb
    image.flags.writeable = False                   # sets the image writeable status to false 
    results = model.process(image)               # Image is actually detected here using mediapipe 
    image.flags.writeable = True                    # Again the image is set it writeable
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)  # color conversion from rgb to bgr
    return image,results
    

In [6]:
def draw_landmarks(image,results):
    
    # Here we are making a customized drawing lines 
    landmark_drawing_spec = mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1)
    connection_drawing_spec = mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
    
    landmark_drawing_spec1 = mp_drawing.DrawingSpec(color=(466,466,466), thickness=1, circle_radius=4)
    connection_drawing_spec1 = mp_drawing.DrawingSpec(color=(000, 000, 000), thickness=1, circle_radius=2)
   
    
    # This will draw the face ,pose and hands connection
    
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_face_mesh.FACEMESH_CONTOURS,
                              landmark_drawing_spec,connection_drawing_spec)
    
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                              landmark_drawing_spec1,connection_drawing_spec1)
    
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              landmark_drawing_spec1,connection_drawing_spec1)
    
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                               landmark_drawing_spec1,connection_drawing_spec1)

original basic code 

In [7]:
# cap = cv2.VideoCapture(0)
# with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    
#     # min_detection - initially the holistic will detect the image with initial detection confidence
#     # min_tracking_confidence - holistic will detect the image with after initial  detection confidence
    
    
#     while cap.isOpened():

#         # after capturing the video in real time the read() function return 2 values - 1 is return value and 2nd is the frame 
#         res,frame = cap.read()

#         # Using a function to make detection 
#         image,results = mediapipe_detection(frame,holistic)

#         # Draw landmarks in real time
#         draw_landmarks(image,results)
        
#         cv2.imshow('Opencv_video',image)
#         # Breaking our infinite loop of capturing the video
    
#         if cv2.waitKey(10) & 0xff == ord('q'):  # pressing q on keyboard will quite from video 
#             break

# cap.release()
# cv2.destroyAllWindows()

In [8]:
# results.face_landmarks
# Here the x and y are the x and y axis position and z will be the relatice distance from camera to object

In [9]:
# plt.imshow(cv2.cvtColor(frame ,cv2.COLOR_BGR2RGB)) # converted to original form

# Section 3 : Extracting the Keypoint Values

In [10]:

# Here we are storing the x,y and z cordinates of pose 
# Flatten function :  is used to convert a multi-dimensional data structure (e.g., a list of lists or an array) into a 1D structure.

def extracted_values(results):
    pose = np.array([[res.x,res.y,res.z,res.visibility]for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)

    face = np.array([[res.x,res.y,res.z]for res in results.face_landmarks.landmark]).flatten()if results.face_landmarks else np.zeros(468*3)

    left_hand = np.array([[res.x,res.y,res.z]for res in results.left_hand_landmarks.landmark]).flatten()if results.left_hand_landmarks else np.zeros(21*3)

    right_hand = np.array([[res.x,res.y,res.z]for res in results.right_hand_landmarks.landmark]).flatten()if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,left_hand,right_hand])


In [51]:
# Saving the each frames in numpy array
result_test = extracted_values(results)
np.save('0',result_test)

# Section 4 : Setup Folders for collection 


In [12]:
# creating the folders 

Data_path = os.path.join("MP_DATA")   #  ensures the correct folder structure is created on your operating system.
actions = np.array(['hello','thanks','namaste'])
no_squences = 30  
sequence_length = 30  # containing the video of 30 frames 

for action in actions:
    for sequence in range(no_squences):
        try:
            os.makedirs(os.path.join(Data_path,action,str(sequence)))
        except:
            pass

# Section 5 : Collect Keypoints Values for Training and Testing 

Use this code for training and adding another actions 

In [85]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    
    # min_detection - initially the holistic will detect the image with initial detection confidence
    # min_tracking_confidence - holistic will detect the image with after initial  detection confidence
    for action in actions:
        for sequence in range(no_squences):
            for frame_num in range(sequence_length):
                
                # after capturing the video in real time the read() function return 2 values - 1 is return value and 2nd is the frame 
                res,frame = cap.read()

                # Using a function to make detection 
                image,results = mediapipe_detection(frame,holistic)
                
                # Draw landmarks in real time
                draw_landmarks(image,results)
                
                if frame_num == 0:
                    cv2.putText(image,'STARTING COLLECTION',(120,200),
                                cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
                    cv2.putText(image,'Collecting frames for {} video number{} '.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image,'Collecting frames for {} video number '.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                
                # Export keypoints
                                    
                keypoints = extracted_values(results)
                npy_path = os.path.join(Data_path,action,str(sequence),str(frame_num))
                np.save(npy_path,keypoints)
                
                cv2.imshow('Opencv_video',image)
                # Breaking our infinite loop of capturing the video
            
                if cv2.waitKey(10) & 0xff == ord('q'):  # pressing q on keyboard will quite from video 
                    break

    cap.release()
    cv2.destroyAllWindows()

# Section 6 : Preprocess Data and Create Labels and Features

In [86]:
# importing libraries
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical 

In [87]:

label_map = {label:num for num,label in enumerate(actions)}
label_map

{'hello': 0, 'thanks': 1, 'namaste': 2}

In [88]:
# Looping throught each frame and adding it to array

sequences,labels = [],[]
for action in actions:
    for sequence in range(no_squences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(Data_path,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
        

In [89]:
# Defining x and y variables for training model
x = np.array(sequences)
y = to_categorical(labels).astype(int)

In [90]:
x.shape
# Here 60 videos with 30 frames each and 1662 keypoints 

(90, 30, 1662)

In [91]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.05)

# Section 7 : Build and Train LSTM Neural Network

In [92]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard

In [93]:
log_dir = os.path.join("Logs")
tb_callback = TensorBoard(log_dir=log_dir)

In [94]:
model = Sequential()  # object of Squential class

# Addeding  a neural network layers and returinig to other layers 
 
model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(30,1662))) 
model.add(LSTM(128,return_sequences=True,activation='relu'))
model.add(LSTM(64,return_sequences=False,activation='relu'))

# not returing to layer because the other next layer is the dense layer 
# the number specified - 64 ,128 and etc are the units 
# input shape is our x data shape with 30 frames and 1662 keypoints 
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))


In [95]:
# Compiling our model 
# We have  multiclass classification therefore use loss='categor...
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [96]:
model.fit(xtrain,ytrain,epochs=2000,callbacks=tb_callback)

Epoch 1/2000
3/3 [==============================] - 4s 152ms/step - loss: 3.6706 - categorical_accuracy: 0.4353
Epoch 2/2000
3/3 [==============================] - 0s 130ms/step - loss: 5.7126 - categorical_accuracy: 0.3529
Epoch 3/2000
3/3 [==============================] - 0s 133ms/step - loss: 3.8266 - categorical_accuracy: 0.2588
Epoch 4/2000
3/3 [==============================] - 1s 191ms/step - loss: 4.0265 - categorical_accuracy: 0.3059
Epoch 5/2000
3/3 [==============================] - 1s 176ms/step - loss: 1.5023 - categorical_accuracy: 0.3882
Epoch 6/2000
3/3 [==============================] - 1s 162ms/step - loss: 1.3863 - categorical_accuracy: 0.1765
Epoch 7/2000
3/3 [==============================] - 1s 182ms/step - loss: 1.1287 - categorical_accuracy: 0.2706
Epoch 8/2000
3/3 [==============================] - 1s 161ms/step - loss: 1.1215 - categorical_accuracy: 0.3176
Epoch 9/2000
3/3 [==============================] - 0s 153ms/step - loss: 1.0929 - categorical_accuracy:

KeyboardInterrupt: 

In [97]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596675 (2.28 MB)
Trainable params: 59667

# Section 8 : Make Predictios

In [98]:
res = model.predict(xtest)

1/1 [==============================] - 0s 258ms/step


# Section 9 : Save Weights


In [99]:
model.save('action.h5')

c:\Users\thomb\anaconda3\envs\Base1\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Section 10 : Evalution using Confusion Matrix and Accuracy

In [100]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score
yhat = model.predict(xtrain)


3/3 [==============================] - 0s 60ms/step


In [101]:
ytrue = np.argmax(ytrain,axis=1).tolist()
ytrue

[2,
 1,
 0,
 2,
 2,
 0,
 0,
 2,
 1,
 1,
 0,
 1,
 0,
 2,
 2,
 2,
 1,
 1,
 1,
 0,
 2,
 1,
 1,
 0,
 1,
 1,
 2,
 2,
 2,
 1,
 0,
 2,
 0,
 1,
 1,
 0,
 2,
 0,
 2,
 1,
 1,
 1,
 0,
 2,
 0,
 0,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 2,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 2,
 0,
 1,
 0,
 2,
 2,
 0,
 2,
 0,
 1,
 1,
 2,
 0,
 0,
 2,
 1,
 2,
 2,
 1,
 2,
 0,
 1,
 1,
 0]

In [102]:
yhat = np.argmax(yhat,axis=1).tolist()
yhat

[2,
 1,
 0,
 2,
 2,
 0,
 0,
 2,
 1,
 1,
 0,
 1,
 0,
 2,
 2,
 2,
 1,
 1,
 1,
 0,
 2,
 1,
 1,
 0,
 1,
 1,
 2,
 2,
 2,
 1,
 0,
 2,
 0,
 1,
 1,
 0,
 2,
 0,
 2,
 1,
 1,
 1,
 0,
 2,
 0,
 0,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 2,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 2,
 0,
 1,
 0,
 2,
 2,
 0,
 2,
 0,
 1,
 1,
 2,
 0,
 0,
 2,
 1,
 2,
 2,
 1,
 2,
 0,
 1,
 0,
 0]

In [103]:
multilabel_confusion_matrix(ytrue,yhat)

array([[[57,  1],
        [ 0, 27]],

       [[56,  0],
        [ 1, 28]],

       [[56,  0],
        [ 0, 29]]], dtype=int64)

In [104]:
accuracy_score(ytrue,yhat)*100

98.82352941176471

# Section 11 : Test in real time

In [105]:
colors = [(245,117,16),(117,245,16),(16,117,245)]
def prob_vis(res1,actions,input_frame,colors):
    output_frame = input_frame.copy()
    for num ,prob in enumerate(res1):
        cv2.rectangle(output_frame,(0,60+num*40),(int(prob*100),90+num*40),colors[num],-1)
        cv2.putText(output_frame,actions[num],(0,85+num*40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
    return output_frame

In [65]:
# plt.imshow(prob_vis(res1,actions,image,colors))

In [108]:
res1 = []
sequence = []
sentence = []
predictions = []
threshold = 0.7

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    
    # min_detection - initially the holistic will detect the image with initial detection confidence
    # min_tracking_confidence - holistic will detect the image with after initial  detection confidence
    
    while cap.isOpened():

        # after capturing the video in real time the read() function return 2 values - 1 is return value and 2nd is the frame 
        res,frame = cap.read()

        # Using a function to make detection 
        image,results = mediapipe_detection(frame,holistic)

        # Draw landmarks in real time
        draw_landmarks(image,results)
        
        
        # Collet the values from already trained data from the disk
        
        
        
    
        
        # Prediction logic
        
        keypoints = extracted_values(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res1 = model.predict(np.expand_dims(sequence,axis=0))[0]
            print(actions[np.argmax(res1)])        
            predictions.append(np.argmax(res))
            
        # Visual logic 
        if np.unique(predictions[-10:]) == np.argmax(res):
            if res1[np.argmax(res1)] > threshold:
                if len(sentence)>0:
                    if actions[np.argmax(res1)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res1)])
                else:
                    sentence.append(actions[np.argmax(res1)])
                
        if len(sentence) > 5:
            sentence = sentence[-5:]
        
        image = prob_vis(res1,actions,image,colors)
        
        cv2.rectangle(image,(0,0),(640,40),(245,117,16),-1)
        cv2.putText(image,' '.join(sentence),(3,30),
                    cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
        

        # Showing to screen    
        cv2.imshow('Opencv_video',image)
        # Breaking our infinite loop of capturing the video
    
        if cv2.waitKey(10) & 0xff == ord('q'):  # pressing q on keyboard will quite from video 
            break

cap.release()
cv2.destroyAllWindows()

C:\Users\thomb\AppData\Local\Temp\ipykernel_10608\5678894.py:37: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if np.unique(predictions[-10:]) == np.argmax(res):


1/1 [==============================] - 0s 35ms/step
hello
1/1 [==============================] - 0s 31ms/step
hello
1/1 [==============================] - 0s 28ms/step
hello
1/1 [==============================] - 0s 29ms/step
hello
1/1 [==============================] - 0s 32ms/step
hello
1/1 [==============================] - 0s 28ms/step
hello
1/1 [==============================] - 0s 31ms/step
hello
1/1 [==============================] - 0s 29ms/step
hello
1/1 [==============================] - 0s 29ms/step
hello
1/1 [==============================] - 0s 28ms/step
hello
1/1 [==============================] - 0s 30ms/step
hello
1/1 [==============================] - 0s 29ms/step
hello
1/1 [==============================] - 0s 28ms/step
hello
1/1 [==============================] - 0s 29ms/step
hello
1/1 [==============================] - 0s 27ms/step
hello
1/1 [==============================] - 0s 29ms/step
hello
1/1 [==============================] - 0s 28ms/step
hello
1/1 [=========